# Notebook Import

In [1]:
import pandas as pd
import numpy as np 

# Constants 

In [82]:
TRAINING_DATA_FILE = 'SpamData/02_Training/train_data.txt'
TEST_DATA_FILE = 'SpamData/02_Training/test_data.txt'


TOKEN_SPAM_PROB_FILE = 'SpamData/03_Testing/prob-spam.txt'
TOKEN_HAM_PROB_FILE = 'SpamData/03_Testing/prob-nonspam.txt'
TOKEN_ALL_PROB_FILE = 'SpamData/03_Testing/prob-all-tokens.txt'


TEST_FEATURE_MATRIX = 'SpamData/03_Testing/test-features.txt'
TEST_TARGET_FILE = 'SpamData/03_Testing/test-target.txt'


VOCAB_SIZE=2500

# Read and Load Features from .txt Files into NunPy Array

In [3]:
sparse_train_data = np.loadtxt(TRAINING_DATA_FILE, delimiter = ' ', dtype=int)

In [4]:
sparse_test_data = np.loadtxt(TEST_DATA_FILE, delimiter =' ', dtype = int) 

In [5]:
sparse_train_data[:5]

array([[ 0,  0,  1,  2],
       [ 0,  7,  1,  1],
       [ 0, 16,  1,  1],
       [ 0, 18,  1,  1],
       [ 0, 24,  1,  1]])

In [6]:
sparse_train_data[-5:]

array([[5795, 1832,    0,    1],
       [5795, 1890,    0,    2],
       [5795, 2125,    0,    1],
       [5795, 2265,    0,    1],
       [5795, 2448,    0,    1]])

In [7]:
print('NR OF ROWS IN TRAINING FILE', sparse_train_data[0])
print('NR OF ROWS IN TEST FILE', sparse_test_data[0])

NR OF ROWS IN TRAINING FILE [0 0 1 2]
NR OF ROWS IN TEST FILE [8 7 1 1]


In [8]:
print('Nr of emails in training file', np.unique(sparse_train_data[:, 0]).size) ##[:, 0]select all the rows in the first column 

Nr of emails in training file 4014


In [9]:
print('Nr of emails in test file', np.unique(sparse_test_data[:, 0]).size)

Nr of emails in test file 1723


# How to create an Empty DataFrame

In [10]:
column_names = ['DOC_D'] + ['Category'] +list(range(0, VOCAB_SIZE))
column_names[:5]

['DOC_D', 'Category', 0, 1, 2]

In [11]:
len(column_names)

2502

In [12]:
index_names = np.unique(sparse_train_data[:,0])
index_names

array([   0,    1,    2, ..., 5791, 5794, 5795])

In [13]:
full_train_data = pd.DataFrame(index=index_names, columns = column_names)

In [14]:
full_train_data.head()

,DOC_D,Category,0,1,2,3,4,5,6,7,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
##set default of 0 rather than Nan 

full_train_data.fillna(value=0, inplace=True)

In [16]:
full_train_data

,DOC_D,Category,0,1,2,3,4,5,6,7,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5789,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5790,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5791,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5794,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Create a Full Matrix from a Sparse Matrix

In [84]:
def make_full_matrix(sparse_matrix, nr_words, doc_idx=0, word_idx=1, cat_idx=2, freq_idx=3):
    """
    Form a full matrix from a sparse matrix. Return a pandas dataframe. 
    Keyword arguments:
    sparse_matrix -- numpy array
    nr_words -- size of the vocabulary. Total number of tokens. 
    doc_idx -- position of the document id in the sparse matrix. Default: 1st column
    word_idx -- position of the word id in the sparse matrix. Default: 2nd column
    cat_idx -- position of the label (spam is 1, nonspam is 0). Default: 3rd column
    freq_idx -- position of occurrence of word in sparse matrix. Default: 4th column
    """
    column_names = ['DOC_D'] + ['CATEGORY'] + list(range(0, VOCAB_SIZE))
    doc_id_names = np.unique(sparse_matrix[:, 0])
    full_matrix = pd.DataFrame(index=doc_id_names, columns=column_names)
    full_matrix.fillna(value=0, inplace=True)
    
    for i in range(sparse_matrix.shape[0]):
        doc_nr = sparse_matrix[i][doc_idx]
        word_id = sparse_matrix[i][word_idx]
        label = sparse_matrix[i][cat_idx]
        occurrence = sparse_matrix[i][freq_idx]
        
        full_matrix.at[doc_nr, 'DOC_D'] = doc_nr
        full_matrix.at[doc_nr, 'CATEGORY'] = label
        full_matrix.at[doc_nr, word_id] = occurrence
    
    full_matrix.set_index('DOC_D', inplace=True)
    return full_matrix
    

In [87]:
%%time
full_train_data = make_full_matrix(sparse_train_data, VOCAB_SIZE)

CPU times: user 7.29 s, sys: 57.5 ms, total: 7.35 s
Wall time: 7.36 s


In [86]:
full_train_data.head()

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_D,,,,,,,,,,,,,,,,,,,,,
0,1,2,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,1,2,0,1,2,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,2,0,2,0,0,3,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,3,0,0,1,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,1,2,4,2,3,1,...,0,0,0,0,0,0,0,0,0,0


# Training the NB MOdel

# Calculating the Probabilty of Spam 

In [27]:
full_train_data.CATEGORY.size

4014

In [29]:
full_train_data.CATEGORY.sum() #--> SPAM = 1, HAM = 0, 

1249

In [30]:
prob_spam = full_train_data.CATEGORY.sum() / full_train_data.CATEGORY.size
print('Proability of spam is', prob_spam)

Proability of spam is 0.3111609367214748


# Total numb of Words/Tokens

In [31]:
#-->pick all the columns expect the category columns
full_train_features = full_train_data.loc[:, full_train_data.columns != 'CATEGORY'] 
full_train_features.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_D,,,,,,,,,,,,,,,,,,,,,
0,2,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,1,2,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,2,0,0,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,1,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,2,4,2,3,1,1,...,0,0,0,0,0,0,0,0,0,0


In [32]:
email_lengths = full_train_features.sum(axis=1) #ths will sum up our columns
email_lengths.shape

(4014,)

In [33]:
email_lengths[:5]

DOC_D
0     50
1     76
2     87
3     76
4    137
dtype: int64

In [34]:
total_wc = email_lengths.sum()
total_wc

444956

# Number of tokens in Spam and Ham Emails

In [35]:
spam_lengths = email_lengths[full_train_data.CATEGORY ==1]
spam_lengths.shape

(1249,)

In [37]:
spam_wc = spam_lengths.sum()
spam_wc

193964

In [38]:
ham_lengths = email_lengths[full_train_data.CATEGORY == 0]

In [39]:
email_lengths.shape[0] - spam_lengths.shape[0] - ham_lengths.shape[0]

0

In [40]:
nonspam_wc = ham_lengths.sum()
nonspam_wc

250992

In [42]:
##check
spam_wc +nonspam_wc -total_wc

0

In [ ]:
#which are longer, the spam or ham emails on average? 

In [44]:
print('Av number of words in SPAM emails: {:.0f}'.format(spam_wc / spam_lengths.shape[0]))
print('Av number of words in HAM emails: {:.3f}'.format(nonspam_wc / ham_lengths.shape[0]))

Av number of words in SPAM emails: 155
Av number of words in HAM emails: 90.775


# Summing the Tokens Occurring in Spam & Ham 

In [45]:
full_train_features.shape

(4014, 2500)

In [46]:
train_spam_tokens = full_train_features.loc[full_train_data.CATEGORY ==1]
train_spam_tokens.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_D,,,,,,,,,,,,,,,,,,,,,
0,2,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,1,2,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,2,0,0,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,1,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,2,4,2,3,1,1,...,0,0,0,0,0,0,0,0,0,0


In [47]:
train_spam_tokens.shape

(1249, 2500)

In [48]:
summed_spam_tokens = train_spam_tokens.sum(axis = 0) +1 ##-->Laplase smoothing technique 
summed_spam_tokens.shape

(2500,)

In [50]:
train_ham_tokens = full_train_features.loc[full_train_data.CATEGORY ==0]
summed_ham_tokens = train_ham_tokens.sum(axis=0)+1

In [51]:
summed_ham_tokens.shape

(2500,)

In [52]:
summed_ham_tokens.tail()

2495    11
2496    20
2497    21
2498    26
2499    38
dtype: int64

In [53]:
train_ham_tokens[2499].sum()+1

38

# P(Token | Spam) - Probability that a Token Occurs given the Email is Spam

In [55]:
prob_tokens_spam = summed_spam_tokens / (spam_wc + VOCAB_SIZE)
prob_tokens_spam[:5]

0    0.009065
1    0.004927
2    0.006882
3    0.010664
4    0.006917
dtype: float64

In [56]:
prob_tokens_spam.sum()

1.0

# P(Token | Ham) - Probability that a Token Occurs given the Email is Nonspam

In [62]:
prob_tokens_nonspam = summed_ham_tokens / (nonspam_wc +VOCAB_SIZE)

In [63]:
prob_tokens_nonspam.sum()

1.0

# P(Token) - Probability that Token Occurs


In [72]:
prob_tokens_all = full_train_features.sum(axis=0) / total_wc

In [73]:
prob_tokens_all.sum()

1.0

# Save the Trained Model

In [75]:
np.savetxt(TOKEN_SPAM_PROB_FILE, prob_tokens_spam)
np.savetxt(TOKEN_HAM_PROB_FILE, prob_tokens_nonspam)
np.savetxt(TOKEN_ALL_PROB_FILE, prob_tokens_all)

# Prepare the Test Data 

In [76]:
sparse_test_data.shape

(110575, 4)

In [78]:
%%time

full_test_data = make_full_matrix(sparse_test_data, nr_words=VOCAB_SIZE)

CPU times: user 3.18 s, sys: 34.4 ms, total: 3.21 s
Wall time: 3.2 s


In [80]:
X_test = full_test_data.loc[:, full_test_data.columns != 'CATEGORY']
y_test = full_test_data.CATEGORY

In [83]:
np.savetxt(TEST_TARGET_FILE, y_test)
np.savetxt(TEST_FEATURE_MATRIX, X_test)